In [9]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
galaxy_train_path = "C:/Users/Poll/Documents/ML project/Data/mdm/mdm_images/galaxy_postage/training"
galaxy_test_path = "C:/Users/Poll/Documents/ML project/Data/mdm/mdm_images/galaxy_postage/test"
star_train_path = "C:/Users/Poll/Documents/ML project/Data/mdm/mdm_images/star_postage/training"
star_test_path = "C:/Users/Poll/Documents/ML project/Data/mdm/mdm_images/star_postage/test"
"Adding features blurring, pixelation and noise"

def load_data(data_path):
    data = []
    for filename in os.listdir(data_path):
        if filename.endswith(".png"):
            img = cv2.imread(os.path.join(data_path, filename), cv2.IMREAD_GRAYSCALE)
            data.append(img.flatten()) # flatten 2D matrix into 1D array
    return np.array(data)
# load data
galaxy_train_data = load_data(galaxy_train_path)
galaxy_test_data = load_data(galaxy_test_path)
star_train_data = load_data(star_train_path)
star_test_data = load_data(star_test_path)
#galaxy_train_data = add_features(galaxy_train_data)
#galaxy_test_data = add_features(galaxy_test_data)
#star_train_data = add_features(star_train_data)
#star_test_data = add_features(star_test_data)

X_train = np.column_stack([galaxy_train_data, star_train_data])
y1_train = np.asarray(pd.read_csv("C:/Users/Poll/Documents/ML project/Data/mdm/mdm_training_solution_sorted.csv")["e1"]).astype('float32')
y2_train = np.asarray(pd.read_csv("C:/Users/Poll/Documents/ML project/Data/mdm/mdm_training_solution_sorted.csv")["e2"]).astype('float32')
X_test = np.column_stack([galaxy_test_data, star_test_data])
y1_test=np.asarray(pd.read_csv("C:/Users/Poll/Documents/ML project/Data/mdm/mdm_training_solution.csv")["e1"]).astype('float32')
y2_test=np.asarray(pd.read_csv("C:/Users/Poll/Documents/ML project/Data/mdm/mdm_training_solution.csv")["e2"]).astype('float32')

In [ ]:
alphas = [0.01, 0.05, 0.1, 0.2, 0.3, 1.0, 10.0, 100.0, 300.0, 500.0, 1e3, 1.5e3, 2e3, 5e3, 1e4]
coef_norm_stored_1 = []
num_nonzero_coefs_stored_1 = []
rmse_stored_1 = []
for alpha in alphas:
    ridge = Ridge(alpha=alpha, normalize=False)
    ridge.fit(X_train, y1_train)
    coef_norm_stored_1.append(np.linalg.norm(ridge.coef_))
    num_nonzero_coefs_stored_1.append(np.count_nonzero(ridge.coef_))
    y1_pred = ridge.predict(X_test)
    rmse_stored_1.append(mean_squared_error(y1_test, y1_pred[0:40000], squared=False))

In [74]:
rmse_stored_1

[0.16853876697128045,
 0.16853876632160086,
 0.16853876550950156,
 0.16853876388530323,
 0.16853876226110548,
 0.1685387508917363,
 0.16853860471649995,
 0.16853714320562527,
 0.16853389697484758,
 0.1685306529093444,
 0.16852255220381435,
 0.1685144649818259,
 0.16850639120821956,
 0.16845822902925395,
 0.16837901159540242]

In [73]:
# Add blurring, pixelation, and noise features
#def add_features(data):
#    features = []
#    for i in range(len(data)):
#        # reshape to 36x64
#        b = data.reshape(9600, 9600)
#        img=b
#        # crop center part to get 32x32
#        #c = b[2:34, 16:48]
#        #img = c[i].reshape(16, 16) # resize to a smaller image to prevent overfitting (original is 48x48 pixels)
#        # Add blurring feature
#        blur = cv2.GaussianBlur(img, (5, 5), 0) # chosen kernel size 5x5 pixels
#        blur_feature = blur.flatten()
#        # Add pixelation feature
#        resized = cv2.resize(img, (10, 10), interpolation=cv2.INTER_NEAREST) # chosen kernel size 10x10 pixels
#        pixel_feature = resized.flatten()
#        # Add noise feature
#        noise = np.random.normal(0, 1, img.shape)
#        noisy_img = img + noise
#        noisy_img[noisy_img < 0] = 0 # min
#        noisy_img[noisy_img > 255] = 255 # max
#        noise_feature = noisy_img.flatten()
#        # Combine all features
#        features.append(np.concatenate([data[i], blur_feature, pixel_feature, noise_feature]))
#    return np.array(features)

[0.16853876697128045,
 0.16853876632160086,
 0.16853876550950156,
 0.16853876388530323,
 0.16853876226110548,
 0.1685387508917363,
 0.16853860471649995,
 0.16853714320562527,
 0.16853389697484758,
 0.1685306529093444,
 0.16852255220381435,
 0.1685144649818259,
 0.16850639120821956,
 0.16845822902925395,
 0.16837901159540242]